In [1]:
import pandas as pd
import urllib2

In [2]:
## Reading the file with the CIKs of S&P 500 companies
df = pd.read_csv('SP_500.csv')

In [3]:
## Creating the CIK with proper format
df['CIK'] = df['CIK'].apply(lambda x: '{0:0>10}'.format(x))

In [4]:
proper_urls_unq = []

In [5]:
for x in df.CIK:
    proper_urls_unq.append("https://www.sec.gov/cgi-bin/browse-edgar?CIK="+ x +"&action=getcompany&type=10-K")

In [6]:
proper_urls_unq

['https://www.sec.gov/cgi-bin/browse-edgar?CIK=0000066740&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0000001800&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0001551152&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0001467373&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0000718877&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0001144215&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0000796343&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0000002488&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0001158449&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0000874761&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=0001122304&action=getcompany&type=10-K',
 'https://www.sec.gov/cgi-bin/br

In [7]:
## Now getting the actual data from each of the proper urls

# Lists to populate
weblink = []
filing_type = []
company_name = []
date = []
url_check = []


for i in range(0,len(proper_urls_unq)):
    
    html = urllib2.urlopen(proper_urls_unq[i]).read()
    
    ## Getting the company name
    html = html[html.find('<span class=\"companyName\">')+10:]
    
    n_start = html.find(">")+1
    n_end = html.find("<")
    name = html[n_start:n_end]
    
    ## getting other values
    while html.find('<td nowrap=\"nowrap\">') != -1:
        
        # Getting loser to the table
        index = html.find('<td nowrap=\"nowrap\">')
        html = html[index+1:]
        
        # Link type
        t_start = html.find(">")+1
        t_end = html.find("<")
        ltype = html[t_start:t_end]
        
        # webLink
        l_start = html.find("<a href=\"/Archives")+9
        l_end = html.find('id=\"documentsbutton\"')-2
        link = "https://www.sec.gov"+html[l_start:l_end]
        
        # date
        d_start = html.find("<td>")+4
        d_end = d_start + 10
        date_val = html[d_start:d_end]
        
        # Appending each row's data into the lists
        # if ltype == "10-K":  ##### This condition can be used to get only 10-K links 
                               ##### However, I felt it is better to get all the data and then filter using pandas 
        weblink.append(link)
        filing_type.append(ltype)
        company_name.append(name.replace("&amp;","&"))
        date.append(date_val)
        url_check.append(proper_urls_unq[i])
        
        ## getting to end of each row
        index = html.find("</tr>")
        html = html[index:]

In [8]:
df_out = pd.DataFrame({'Company Name': company_name,'Date of filing': date,'Filing type': filing_type,'weblink': weblink})

In [9]:
df_out.to_csv('10K Links Project.csv',index = False)